<a href="https://colab.research.google.com/github/aymnblh/AI/blob/main/Restaurant_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot  as plt
import numpy as np
import seaborn as sns
import string
import nltk as nk
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import words
from sklearn.model_selection import train_test_split
import nltk

In [ ]:
data=pd.read_csv("/content/data_2_train.csv",sep = ',')
data.head()

,example_id,text,aspect_term,term_location,class
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0


In [ ]:
data2=pd.read_csv("/content/test_c.csv",sep = ';')
data2.head()

,text,aspect_term,class
0,the service was really bad but the chips were ...,service,-1
1,the service was really bad but the chips were ...,chips,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3602 entries, 0 to 3601
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   example_id      3602 non-null   object
 1    text           3602 non-null   object
 2    aspect_term    3602 non-null   object
 3    term_location  3602 non-null   object
 4    class          3602 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 140.8+ KB


In [ ]:
data.groupby([' class'])[' text'].count()

 class
-1     805
 0     633
 1    2164
Name:  text, dtype: int64

In [ ]:
data["phrase_len"]= data[" text"].str.len() 
data.head()

,example_id,text,aspect_term,term_location,class,phrase_len
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1,36
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1,169
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1,165
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1,165
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0,165


In [ ]:
data.groupby([' class'])['phrase_len'].mean()

 class
-1    113.647205
 0    106.360190
 1     96.292976
Name: phrase_len, dtype: float64

In [ ]:
data.loc[data['phrase_len']<3,' text']='Null'

In [ ]:
data.pop("phrase_len")

0        36
1       169
2       165
3       165
4       165
       ... 
3597    171
3598    171
3599    171
3600    171
3601    171
Name: phrase_len, Length: 3602, dtype: int64

In [ ]:
data['Review']=data[' text']
data.pop(" text")
data['Liked']=data[' class']
data.pop(" class")
data.head()

,example_id,aspect_term,term_location,Review,Liked
0,3121_0,staff,8--13,But the staff was so horrible to us.,-1
1,2777_0,food,57--61,To be completely fair[comma] the only redeemin...,1
2,1634_0,food,4--8,The food is uniformly exceptional[comma] with ...,1
3,1634_1,kitchen,55--62,The food is uniformly exceptional[comma] with ...,1
4,1634_2,menu,141--145,The food is uniformly exceptional[comma] with ...,0


In [ ]:
import string
data['Review'] = data['Review'].apply(lambda x: x.replace('[comma]',',').lower())
string.punctuation
def remove_punctuation(txt):
    txt_nopunt="".join([c for c in txt if c not in string.punctuation])
    return txt_nopunt
data ['Review'] = data ['Review'].apply(lambda x: remove_punctuation(x))
data ['Review'] = data ['Review']. str.replace ('br' , '')

data ['Review'] = data ['Review']. str.lower () 
data2 ['Review'] = data2 ['text']. str.lower ()
data.head()

,example_id,aspect_term,term_location,Review,Liked
0,3121_0,staff,8--13,but the staff was so horrible to us,-1
1,2777_0,food,57--61,to be completely fair the only redeeming facto...,1
2,1634_0,food,4--8,the food is uniformly exceptional with a very ...,1
3,1634_1,kitchen,55--62,the food is uniformly exceptional with a very ...,1
4,1634_2,menu,141--145,the food is uniformly exceptional with a very ...,0


In [ ]:
import spacy
lst = data[" aspect_term"].tolist()
nlp = spacy.load('en_core_web_sm', parser=False)
def pos(text):
    doc = nlp(text)
    aspects = [token.text for token in doc if token.pos_ == 'NOUN' ]
    for element in aspects:
     if element not in lst:
       aspects.remove(element)
    return aspects
data["Aspects"] = data["Review"].apply(pos)
data.head(15)

,example_id,aspect_term,term_location,Review,Liked,Aspects
0,3121_0,staff,8--13,but the staff was so horrible to us,-1,[staff]
1,2777_0,food,57--61,to be completely fair the only redeeming facto...,1,"[food, teodora]"
2,1634_0,food,4--8,the food is uniformly exceptional with a very ...,1,"[food, kitchen, menu]"
3,1634_1,kitchen,55--62,the food is uniformly exceptional with a very ...,1,"[food, kitchen, menu]"
4,1634_2,menu,141--145,the food is uniformly exceptional with a very ...,0,"[food, kitchen, menu]"
5,2846_0,food,17--21,not only was the food outstanding but the litt...,1,"[food, perks]"
6,2846_1,perks,51--56,not only was the food outstanding but the litt...,1,"[food, perks]"
7,1458_0,orrechiete with sausage and chicken,27--62,our agreed favorite is the orrechiete with sau...,1,"[sausage, chicken, waiters, dish, meats]"
8,1458_1,waiters,76--83,our agreed favorite is the orrechiete with sau...,1,"[sausage, chicken, waiters, dish, meats]"
9,1458_2,meats,152--157,our agreed favorite is the orrechiete with sau...,0,"[sausage, chicken, waiters, dish, meats]"


In [ ]:
def pos(text):
    doc = nlp(text)
    aspects = [token.text for token in doc if token.pos_ == 'NOUN' ]
    for element in aspects:
     if element not in lst:
       aspects.remove(element)
    return aspects
data2 ["Aspects"] = data2["Review"].apply(pos)

In [ ]:
nltk.download('punkt')
data['Review'] = data.apply(lambda row: nltk.word_tokenize(row['Review']), axis=1)
data[' aspect_term'] = data[' aspect_term'].apply(lambda x: x.lower())
data[' aspect_term'] = data.apply(lambda row: nltk.word_tokenize(row[' aspect_term']), axis=1)
data[' aspect_term'] = data[' aspect_term'].apply(lambda x:" ".join(x))
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,example_id,aspect_term,term_location,Review,Liked,Aspects
0,3121_0,staff,8--13,"[but, the, staff, was, so, horrible, to, us]",-1,[staff]
1,2777_0,food,57--61,"[to, be, completely, fair, the, only, redeemin...",1,"[food, teodora]"
2,1634_0,food,4--8,"[the, food, is, uniformly, exceptional, with, ...",1,"[food, kitchen, menu]"
3,1634_1,kitchen,55--62,"[the, food, is, uniformly, exceptional, with, ...",1,"[food, kitchen, menu]"
4,1634_2,menu,141--145,"[the, food, is, uniformly, exceptional, with, ...",0,"[food, kitchen, menu]"


In [ ]:
nltk.download('punkt')
data2['Review'] = data2.apply(lambda row: nltk.word_tokenize(row['Review']), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('stopwords')
l=["only","but","however","otherwise","also","ain", "aren", "aren't", "couldn", "couldn't", "didn", "didn't", "doesn", "doesn't", "hadn", "hadn't", 'hasn', "hasn't", "haven", "haven't", "isn", "isn't","mightn", "mightn't","mustn", "mustn't", "needn", "needn't", "shan", "shan't", "shouldn", "shouldn't", "wasn", "wasn't", "weren", "weren't", "won't", "wouldn", "wouldn't","no","nor","not","don", "don't"]
pd.set_option('display.max_colwidth', 0)
stop = stopwords.words('english')
for element in l:
    if element in stop:
        stop.remove(element)  
data['Review'] = data['Review'].apply(lambda x:[c for c in x if c not in stop])
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,example_id,aspect_term,term_location,Review,Liked,Aspects
0,3121_0,staff,8--13,"[but, staff, horrible, us]",-1,[staff]
1,2777_0,food,57--61,"[completely, fair, only, redeeming, factor, food, average, but, couldnt, make, deficiencies, teodora]",1,"[food, teodora]"
2,1634_0,food,4--8,"[food, uniformly, exceptional, capable, kitchen, proudly, whip, whatever, feel, like, eating, whether, menu, not]",1,"[food, kitchen, menu]"
3,1634_1,kitchen,55--62,"[food, uniformly, exceptional, capable, kitchen, proudly, whip, whatever, feel, like, eating, whether, menu, not]",1,"[food, kitchen, menu]"
4,1634_2,menu,141--145,"[food, uniformly, exceptional, capable, kitchen, proudly, whip, whatever, feel, like, eating, whether, menu, not]",0,"[food, kitchen, menu]"


In [ ]:
pd.set_option('display.max_colwidth', 0)
stop = stopwords.words('english')
for element in l:
    if element in stop:
        stop.remove(element)  
data2['Review'] = data2['Review'].apply(lambda x:[c for c in x if c not in stop])

In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
data.Review = data.apply(lambda row: [stemmer.stem(item) for item in row.Review], axis=1)
data.head()

,example_id,aspect_term,term_location,Review,Liked,Aspects
0,3121_0,staff,8--13,"[but, staff, horribl, us]",-1,[staff]
1,2777_0,food,57--61,"[complet, fair, onli, redeem, factor, food, averag, but, couldnt, make, defici, teodora]",1,"[food, teodora]"
2,1634_0,food,4--8,"[food, uniform, except, capabl, kitchen, proud, whip, whatev, feel, like, eat, whether, menu, not]",1,"[food, kitchen, menu]"
3,1634_1,kitchen,55--62,"[food, uniform, except, capabl, kitchen, proud, whip, whatev, feel, like, eat, whether, menu, not]",1,"[food, kitchen, menu]"
4,1634_2,menu,141--145,"[food, uniform, except, capabl, kitchen, proud, whip, whatev, feel, like, eat, whether, menu, not]",0,"[food, kitchen, menu]"


In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
data2.Review = data2.apply(lambda row: [stemmer.stem(item) for item in row.Review], axis=1)

In [ ]:
data['aspect_term']=data[' aspect_term']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
data['Review']=data['Review'].astype(str)
v = TfidfVectorizer()
x = v.fit_transform(data['Review'])
print('Tfidf:',x.shape)

Tfidf: (3602, 2946)


In [ ]:
data2['Review']=data2['Review'].astype(str)
v2 = TfidfVectorizer()
x2 = v.fit_transform(data2['Review'])
print('Tfidf:',x.shape)

Tfidf: (3602, 2946)


wordtovec

In [ ]:
#import gensim

In [ ]:
#list=[]
#for i in data.Review:
#     li = list(i.split(""))
#     list.append(li)

In [ ]:
#model = gensim.models.Word2Vec(
#    window=10,
#    min_count=2,
#    workers=4,
#)
#model.build_vocab(data['Review'], progress_per=1000)

In [ ]:
#model.train(data['Review'], total_examples=model.corpus_count, epochs=model.epochs)

doctovec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text
card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data.Review)]
#display the tagged docs
card_docs

[TaggedDocument(words=["['but',", "'staff',", "'horribl',", "'us']"], tags=[0]),
 TaggedDocument(words=["['complet',", "'fair',", "'onli',", "'redeem',", "'factor',", "'food',", "'averag',", "'but',", "'couldnt',", "'make',", "'defici',", "'teodora']"], tags=[1]),
 TaggedDocument(words=["['food',", "'uniform',", "'except',", "'capabl',", "'kitchen',", "'proud',", "'whip',", "'whatev',", "'feel',", "'like',", "'eat',", "'whether',", "'menu',", "'not']"], tags=[2]),
 TaggedDocument(words=["['food',", "'uniform',", "'except',", "'capabl',", "'kitchen',", "'proud',", "'whip',", "'whatev',", "'feel',", "'like',", "'eat',", "'whether',", "'menu',", "'not']"], tags=[3]),
 TaggedDocument(words=["['food',", "'uniform',", "'except',", "'capabl',", "'kitchen',", "'proud',", "'whip',", "'whatev',", "'feel',", "'like',", "'eat',", "'whether',", "'menu',", "'not']"], tags=[4]),
 TaggedDocument(words=["['not',", "'onli',", "'food',", "'outstand',", "'but',", "'littl',", "'perk',", "'great']"], tags=[

In [ ]:
#tokenize and tag the card text
card_docs2 = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data2.Review)]

In [ ]:
model = Doc2Vec(vector_size=64, min_count=2, epochs = 20)

In [ ]:
#instantiate model
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model.build_vocab(card_docs)
#train model
model.train(card_docs, total_examples=model.corpus_count
            , epochs=model.epochs)

In [ ]:
#generate vectors
card2vec = [model.infer_vector((data['Review'][i].split(' '))) 
            for i in range(0,len(data['Review']))]
card2vec

[array([-0.21878904,  0.16872218,  0.03314419,  0.2838511 , -0.03461935,
         0.12264069,  0.24211079, -0.22408251,  0.01182086,  0.05016461,
         0.0353807 ,  0.04485288, -0.04056806, -0.02638022,  0.09834063,
        -0.19583136, -0.05334231, -0.02343446, -0.03286781,  0.02510437,
         0.05832661, -0.01391346,  0.12398666,  0.04842202, -0.06692141,
        -0.14776781, -0.00608472,  0.02521742, -0.20856391, -0.07332433,
        -0.1397433 , -0.26891634,  0.12762596, -0.28465533,  0.314195  ,
         0.3288915 ,  0.03473762, -0.00564871, -0.04727181,  0.00605495,
         0.12153487, -0.0739914 ,  0.26128265, -0.12517494, -0.05365746,
         0.2580919 , -0.11984612,  0.06754113, -0.02344095,  0.02534541,
        -0.05666025,  0.07364953, -0.03574659, -0.10129384,  0.12759517,
        -0.03326391, -0.00822425, -0.12612247,  0.02699398, -0.042604  ,
        -0.06323554,  0.01578984, -0.03915183,  0.25365847], dtype=float32),
 array([ 0.4358446 ,  0.36678016, -0.00909056, 

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text
card_docs3 = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data. aspect_term)]

In [ ]:
#instantiate model
model3 = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model3.build_vocab(card_docs3)
#train model
model3.train(card_docs3, total_examples=model.corpus_count
            , epochs=model.epochs)

In [ ]:
#generate vectors
aspect2vec = [model3.infer_vector((data['aspect_term'][i].split(' '))) 
            for i in range(0,len(data['aspect_term']))]

In [ ]:
#Create a list of lists
dtv3= np.array(aspect2vec).tolist()
#set list to dataframe column
data['aspect2vec'] = dtv3

In [ ]:
#instantiate model
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model.build_vocab(card_docs2)
#train model
model.train(card_docs2, total_examples=model.corpus_count
            , epochs=model.epochs)

In [ ]:
#generate vectors
c2vec = [model.infer_vector((data2['Review'][i].split(' '))) 
            for i in range(0,len(data2['Review']))]

In [ ]:
import numpy as np
#Create a list of lists
dtv2= np.array(c2vec).tolist()
#set list to dataframe column
data2['c2vec'] = dtv2

In [ ]:
import numpy as np
#Create a list of lists
dtv= np.array(card2vec).tolist()
#set list to dataframe column
data['card2vec'] = dtv
data.head(5)

,example_id,aspect_term,term_location,Review,Liked,Aspects,aspect_term,aspect2vec,card2vec
0,3121_0,staff,8--13,"['but', 'staff', 'horribl', 'us']",-1,[staff],staff,"[-0.009033755399286747, -0.018396612256765366, -0.012770210392773151, 0.008716657757759094, 0.0039543407037854195, -0.004171024542301893, 0.02476540207862854, 0.008921679109334946, -0.01434667594730854, -0.005296783987432718, -0.013686553575098515, 0.005680615082383156, -0.007402082905173302, -0.003252983558923006, -0.005822546314448118, 0.008820924907922745, -0.022302083671092987, -0.01062464714050293, 0.0021462617442011833, 0.008469294756650925, -0.004613864701241255, 0.012051829136908054, 0.019750645384192467, -0.004913643933832645, -0.003018269082531333, -0.0053708176128566265, 0.019085969775915146, -0.010808882303535938, 0.0040998333133757114, 0.018064284697175026, -0.01281591597944498, 0.011383206583559513, 0.006301402114331722, -0.009329840540885925, 0.02501804567873478, 0.02396536059677601, 0.011319843120872974, -0.023454250767827034, -0.009478377178311348, 0.007721073925495148, 0.02040177211165428, -0.01566278375685215, 0.00939892791211605, -0.012358728796243668, -0.01145093236118555, -0.024426788091659546, -0.00013811867393087596, -0.006780377123504877, 0.021219562739133835, 0.04149990901350975, -0.026988789439201355, 0.007526411674916744, 0.01139754243195057, 0.0063307033851742744, 0.009853377938270569, 0.014352526515722275, 0.012137820944190025, 0.0002388988359598443, 0.00973372533917427, 0.014434590004384518, 0.016342055052518845, -0.006281706970185041, 0.019012482836842537, 0.004770578816533089]","[-0.21878904104232788, 0.16872218251228333, 0.03314419090747833, 0.28385108709335327, -0.034619349986314774, 0.1226406916975975, 0.24211078882217407, -0.22408251464366913, 0.01182086393237114, 0.0501646064221859, 0.03538070246577263, 0.04485287517309189, -0.04056805744767189, -0.02638022042810917, 0.09834063053131104, -0.19583135843276978, -0.0533423088490963, -0.023434458300471306, -0.03286781162023544, 0.025104371830821037, 0.058326609432697296, -0.013913462869822979, 0.12398666143417358, 0.04842201992869377, -0.0669214054942131, -0.1477678120136261, -0.0060847243294119835, 0.025217415764927864, -0.2085639089345932, -0.07332433015108109, -0.13974329829216003, -0.2689163386821747, 0.1276259571313858, -0.2846553325653076, 0.31419500708580017, 0.32889148592948914, 0.03473762422800064, -0.005648714490234852, -0.047271810472011566, 0.0060549546033144, 0.12153486907482147, -0.07399139553308487, 0.26128265261650085, -0.12517493963241577, -0.05365746095776558, 0.25809189677238464, -0.11984612047672272, 0.06754112988710403, -0.023440953344106674, 0.025345414876937866, -0.056660246104002, 0.07364952564239502, -0.03574659302830696, -0.10129383951425552, 0.12759517133235931, -0.0332639105618, -0.008224252611398697, -0.12612247467041016, 0.026993978768587112, -0.042604003101587296, -0.06323553621768951, 0.01578984409570694, -0.03915182873606682, 0.2536584734916687]"
1,2777_0,food,57--61,"['complet', 'fair', 'onli', 'redeem', 'factor', 'food', 'averag', 'but', 'couldnt', 'make', 'defici', 'teodora']",1,"[food, teodora]",food,"[0.005240345373749733, 0.005220400635153055, 0.012364554218947887, -0.011376024223864079, 0.008463405072689056, 0.0067732264287769794, -0.010798299685120583, -0.0022469046525657177, 0.002168096136301756, 0.0021791821345686913, 0.008213749155402184, -0.009820795617997646, 0.0025684700813144445, 0.002794927451759577, 0.005205606576055288, -0.002888611052185297, 0.01378844678401947, 0.009924042038619518, 0.0021987566724419594, -0.005129185505211353, 0.005915271118283272, -0.0027127303183078766, -0.0020150477066636086, -0.0004795673885382712, -0.0002744127996265888, -0.0026519817765802145, 7.989117875695229e-05, 0.007641656789928675, 0.007355699315667152, -0.01430586352944374, 0.00013608578592538834, 0.004406091757118702, -0.008350882679224014, 0.004661429673433304, -0.011635368689894676, -0.0056569683365523815, -0.008223784156143665, 0.01169937569648

Decision Tree

In [ ]:
x1 = list(data['card2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree   
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#splitting data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=27)  #splitting data with test size of 30%

#applying tree algorithm
tree = tree.DecisionTreeClassifier()  
tree.fit(x_train, y_train)   #fitting our model
y_pred=tree.predict(x_test)   # evaluating our model
print("score:{}".format(accuracy_score(y_test, y_pred)))

score:0.5541165587419057


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.38      0.38      0.38       250
           0       0.30      0.33      0.31       189
           1       0.71      0.69      0.70       642

    accuracy                           0.55      1081
   macro avg       0.46      0.47      0.46      1081
weighted avg       0.56      0.55      0.56      1081



Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier #Importing Random Forest Classifier
from sklearn import metrics  # Importing metrics to test accuracy
clf=RandomForestClassifier(n_estimators=10)  #Creating a random forest with 100 decision trees
clf.fit(x_train, y_train)  #Training our model
y_pred=clf.predict(x_test)  #testing our model
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))  #Measuring the accuracy of our model

Accuracy: 0.6170212765957447


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.43      0.38      0.41       250
           0       0.37      0.18      0.24       189
           1       0.70      0.84      0.76       642

    accuracy                           0.62      1081
   macro avg       0.50      0.47      0.47      1081
weighted avg       0.58      0.62      0.59      1081



Knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Create feature and target arrays
x1 = list(data['card2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
             x, y, test_size = 0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

y_pred=knn.predict(X_test)

print('Accuracy ', accuracy_score(y_pred,y_test))

Accuracy  0.651872399445215


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.49      0.48       161
           0       0.44      0.43      0.44       132
           1       0.78      0.78      0.78       428

    accuracy                           0.65       721
   macro avg       0.57      0.57      0.57       721
weighted avg       0.65      0.65      0.65       721



In [ ]:
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(
             x, y, test_size = 0.2, random_state=42)
model = GaussianNB()
model.fit(x, y);
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))  #Measuring the accuracy of our model

Accuracy: 0.651872399445215


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.49      0.48       161
           0       0.44      0.43      0.44       132
           1       0.78      0.78      0.78       428

    accuracy                           0.65       721
   macro avg       0.57      0.57      0.57       721
weighted avg       0.65      0.65      0.65       721



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#splitting data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=27)  #splitting data with test size of 30%
logreg = LogisticRegression()   #build our logistic model
logreg.fit(x_train, y_train)  #fitting training data
y_pred  = logreg.predict(x_test)    #testing model’s performance
print("Accuracy={:.2f}".format(logreg.score(x_test, y_test)))
print("Accuracy train data={:.2f}".format(logreg.score(x_train, y_train)))

Accuracy=0.64
Accuracy train data=0.66


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.29      0.36       250
           0       0.45      0.20      0.28       189
           1       0.69      0.91      0.78       642

    accuracy                           0.64      1081
   macro avg       0.54      0.47      0.47      1081
weighted avg       0.60      0.64      0.60      1081



In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout
x1 = list(data['card2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)
y=tf.keras.utils.to_categorical(y, 3)
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu')
])
model.add(Dense(64, input_dim=128, activation='relu'))
model.add(Dense(3, input_dim=64, activation='sigmoid'))

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
model.fit(x_train, y_train, epochs=400)


Epoch 1/400
85/85 [==============================] - 1s 2ms/step - loss: 0.8890 - accuracy: 0.6205
Epoch 2/400
85/85 [==============================] - 0s 2ms/step - loss: 0.8046 - accuracy: 0.6512
Epoch 3/400
85/85 [==============================] - 0s 2ms/step - loss: 0.7795 - accuracy: 0.6561
Epoch 4/400
85/85 [==============================] - 0s 2ms/step - loss: 0.7573 - accuracy: 0.6768
Epoch 5/400
85/85 [==============================] - 0s 2ms/step - loss: 0.7378 - accuracy: 0.6860
Epoch 6/400
85/85 [==============================] - 0s 2ms/step - loss: 0.7180 - accuracy: 0.6975
Epoch 7/400
85/85 [==============================] - 0s 2ms/step - loss: 0.7010 - accuracy: 0.7031
Epoch 8/400
85/85 [==============================] - 0s 2ms/step - loss: 0.6797 - accuracy: 0.7112
Epoch 9/400
85/85 [==============================] - 0s 2ms/step - loss: 0.6577 - accuracy: 0.7197
Epoch 10/400
85/85 [==============================] - 0s 2ms/step - loss: 0.6463 - accuracy: 0.7301
Epoch 11/

In [ ]:
model.evaluate(x_test,y_test)


29/29 [==============================] - 0s 1ms/step - loss: 3.6164 - accuracy: 0.6759


[3.616446018218994, 0.6759156584739685]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       163
           1       0.71      0.41      0.52       523
           2       0.27      0.74      0.40       215

    accuracy                           0.42       901
   macro avg       0.33      0.39      0.31       901
weighted avg       0.48      0.42      0.40       901



In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
x1 = list(data['card2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))

y = data['Liked']
y=tf.keras.utils.to_categorical(y, 3)
x = np.array(x)
x = x.reshape(x.shape[0], -1)
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.1,random_state=35)
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu')
])
model.add(Dense(64, input_dim=128, activation='relu'))
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(3, input_dim=64, activation='sigmoid'))
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)
model.fit(x_train, y_train, epochs=300)

Epoch 1/300
102/102 [==============================] - 1s 2ms/step - loss: 0.9150 - accuracy: 0.6020
Epoch 2/300
102/102 [==============================] - 0s 2ms/step - loss: 0.8138 - accuracy: 0.6513
Epoch 3/300
102/102 [==============================] - 0s 2ms/step - loss: 0.7749 - accuracy: 0.6720
Epoch 4/300
102/102 [==============================] - 0s 2ms/step - loss: 0.7432 - accuracy: 0.6865
Epoch 5/300
102/102 [==============================] - 0s 2ms/step - loss: 0.7147 - accuracy: 0.7063
Epoch 6/300
102/102 [==============================] - 0s 2ms/step - loss: 0.6903 - accuracy: 0.7186
Epoch 7/300
102/102 [==============================] - 0s 2ms/step - loss: 0.6587 - accuracy: 0.7269
Epoch 8/300
102/102 [==============================] - 0s 2ms/step - loss: 0.6329 - accuracy: 0.7439
Epoch 9/300
102/102 [==============================] - 0s 2ms/step - loss: 0.6091 - accuracy: 0.7562
Epoch 10/300
102/102 [==============================] - 0s 2ms/step - loss: 0.5777 - accura

In [ ]:
model.evaluate(x_test,y_test)

12/12 [==============================] - 0s 2ms/step - loss: 3.9593 - accuracy: 0.6704


[3.9592697620391846, 0.6703600883483887]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.21      0.92      0.34        71
           1       0.50      0.01      0.02       211
           2       0.27      0.16      0.20        79

    accuracy                           0.22       361
   macro avg       0.33      0.36      0.19       361
weighted avg       0.39      0.22      0.12       361

